### Dados da Saúde do Coração
#### Logistic regression To predict heart disease
Link para a [Base de dados e Documentação Oficial](https://www.kaggle.com/datasets/dileep070/heart-disease-prediction-using-logistic-regression?select=framingham.csv)

#### 1. Preparando o ambiente: base de dados

In [ ]:
import pandas as pd

dados = pd.read_csv('./db-framingham.csv')
dataset = pd.DataFrame(dados)
dataset.head()


In [ ]:
dataset.shape

In [ ]:
dataset.columns

In [ ]:
type(dataset)

In [ ]:
dict_eng_port = {
  'male' : 'genero',
  'age' : 'idade',
  'education' : 'escolaridade',
  'currentSmoker' : 'fumante',
  'cigsPerDay' : 'cigarros_por_dia',
  'BPMeds' : 'uso_medicamento_pressao',
  'prevalentStroke' : 'avc',
  'prevalentHyp' : 'hipertensao',
  'diabetes' : 'diabetes',
  'totChol' : 'colesterol_total',
  'sysBP' : 'pressao_arterial_sistolica',
  'diaBP' : 'pressao_arterial_diastolica',
  'BMI' : 'imc',
  'heartRate' : 'freq_cardiaca',
  'glucose' : 'glicemia',
  'TenYearCHD' : 'risco_dac_decada'
}
dataset = dataset.rename(columns=dict_eng_port)
dataset.head()

In [ ]:
dataset = dataset.drop(columns=['escolaridade'])
dataset.head()

#### 2. Trabalhando com Data Frame

In [ ]:
dataset.describe()

In [ ]:
dataset.dtypes

In [ ]:
dataset.cigarros_por_dia.unique()

In [ ]:
dataset.isnull().sum()

In [ ]:
selecao_corretos = dataset.cigarros_por_dia.notnull()
selecao_corretos

In [ ]:
dataset = dataset[selecao_corretos]
dataset.isnull().sum()

In [ ]:
dataset = dataset[dataset.uso_medicamento_pressao.notnull()]
dataset = dataset[dataset.colesterol_total.notnull()]
dataset = dataset[dataset.imc.notnull()]
dataset = dataset[dataset.freq_cardiaca.notnull()]

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.head()

#### 3. Trabalhando com Dados Faltantes de Glicemia

In [ ]:
dataset.isnull().sum()

In [ ]:
pessoas_com_diabetes = dataset[dataset.diabetes == 1]
pessoas_sem_diabetes = dataset[dataset.diabetes == 0]

In [ ]:
pessoas_com_diabetes.glicemia.head()
# pessoas com diabetes podem ter glicemia tanto alta como baixa glicemia
# abaixo de 70 hipoglicemia
# entre 71 e 110 normal
# entre 111 e 126 pre diabetes
# acima de 127 diabetes

In [ ]:
pessoas_com_diabetes.glicemia.mean()

In [ ]:
pessoas_sem_diabetes.glicemia.mean()

In [ ]:
dataset.glicemia.mean()

In [ ]:
dataset.glicemia.describe()

In [ ]:
dataset.diabetes.value_counts()

In [ ]:
import seaborn as sns

ax = sns.distplot(pessoas_com_diabetes.glicemia, norm_hist=False)
ax.set_xlim(0, 400)

In [ ]:
import seaborn as sns

ax = sns.distplot(pessoas_sem_diabetes.glicemia, norm_hist=False)
ax.set_xlim(0, 400)

In [ ]:
nulos_glicemia = dataset.glicemia.isnull()

In [ ]:
dataset[nulos_glicemia].diabetes.value_counts()
# dos dados sem glicemia 339, 337 não tem diabetes e 2 tem diabetes, então não vale a pena fazer manipulação de dados para 2 pessoas com diabetes
# assim podemos simplesmente descartar as pessoas que não tem dados de glicemia, pois 99% delas são pessoas sem diabetes 

In [ ]:
dataset.glicemia.mean()

In [ ]:
dataset.fillna({'glicemia': 81.88}, inplace=True)

In [ ]:
dataset.isnull().sum()

#### 4. Preparação de Dados - convertendo tipos

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.dtypes

In [ ]:
# converte tipos de dados nos lugares especificados
dataset = dataset.astype({'uso_medicamento_pressao': 'int64'})
dataset.dtypes

In [ ]:
dataset = dataset.astype({'cigarros_por_dia': 'int64'})
dataset.dtypes

In [ ]:
dataset.head()

#### 5. Executando transformações - Categorias

In [ ]:
dataset.cigarros_por_dia.describe()

In [ ]:
ax = sns.distplot(dataset.cigarros_por_dia, norm_hist=False )
ax.set_xlim(0,70)

In [ ]:
dataset.cigarros_por_dia.value_counts()

In [ ]:
dataset.fumante.value_counts()

In [ ]:
sns.boxplot(dataset.cigarros_por_dia)

In [ ]:
dataset.cigarros_por_dia.mean()

In [ ]:
# categorias de fumantes
# 0 = não fumante
# 1 =  fumante leve (1 até 10)
# 2 =  fumante moderado (11 até 20)
# 3 =  fumante severo (mais de 20 cigarros)

limites = [-1, 1, 11, 20, 71]
categorias = [0, 1, 2, 3]

fumante_categorico = pd.cut(dataset.cigarros_por_dia, limites, labels=categorias)

In [ ]:
fumante_categorico

In [ ]:
dataset = dataset.assign(fumante_categorico = fumante_categorico)
dataset.head()

In [ ]:
dataset.columns

#### 6. Visualizando Dados

In [ ]:
dataset.info()

In [ ]:
dataset.plot()

In [ ]:
dataset.plot('glicemia')

In [ ]:
sns.pairplot(dataset)

#### 7. Correlação

In [ ]:
corr = dataset.corr()
corr

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

paleta = sns.color_palette('light:salmon', as_cmap=True)

sns.heatmap(dataset.corr(), annot=True, cmap = paleta)

plt.figure(figsize=[16,12])

####  8. Balanceamento de dados

In [ ]:
dataset.head()

In [ ]:
# podemos realizar um Down Sampling para não criar distorções no Data Frame
# Muitas vezes, quando trabalhamos com dados, existem assimetrias que dificultam o uso de técnicas de Machine Learning para gerar modelos. Normalmente essas assimetrias acontecem quando temos uma grande quantidade de registros em uma categoria, e uma quantidade pequena em outra. Para resolver essa questão, existem diversas técnicas de balanceamento dos dados
# A técnica de Sobre Amostragem Up Sampling cria novos registros baseados nos registros existentes no dataset, porém pode gerar distorções no Data Frame
# dataset.to_csv('./dados_saude_final.csv', index=False)